In [ ]:
import pandas as pd
import numpy as np
import joblib

# Load model
best_model = joblib.load("../../models/best_severity_model.pkl")

# Feature engineering
def add_features(df):
    df["TransactionMonth"] = pd.to_datetime(df["TransactionMonth"], errors="coerce")
    df["PolicyAgeDays"] = (pd.Timestamp.now() - df["TransactionMonth"]).dt.days
    df["VehicleAge"] = df["TransactionMonth"].dt.year - df["RegistrationYear"]
    df["PremiumToSumInsured"] = df["TotalPremium"] / df["SumInsured"]
    return df

# Load new data
df = pd.read_csv("../../data/processed/processed_insurance_data.csv")
df = add_features(df)

# Severity subset
prediction_df = df[df["TotalClaims"] > 0].copy()

# Features only
X_new = prediction_df.drop(columns=["TotalClaims"])

# Predict
pred = best_model.predict(X_new)

# Attach to dataframe
prediction_df["PredictedClaims"] = pred

print(prediction_df[["TotalClaims", "PredictedClaims"]].head())
